In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sos4hjb.polynomials import Variable, MonomialVector, ChebyshevVector, Polynomial
from sos4hjb.optimization.cvx import SosProgram

# System dynamics

In [ ]:
# State with limits.
x = Variable('x')
xlim = 1
x_m = MonomialVector.make_polynomial(x)
xlim_m = MonomialVector.make_polynomial(xlim)
X = (x_m + xlim_m) * (xlim_m - x_m)

# Input with limits.
u = Variable('u')
ulim = 1
u_m = MonomialVector.make_polynomial(u)
ulim_m = MonomialVector.make_polynomial(ulim)
U = (u_m + ulim_m) * (ulim_m - u_m)

# Scalar dynamics.
fx = x_m - 4 * x_m ** 3
fu = MonomialVector.make_polynomial(1)
f = fx + fu * u_m

# Running cost.
Q = 1
R = 1
lx = Q * x_m ** 2
lu = R * u_m ** 2
l = lx + lu

In [ ]:
# Utility function to plot polynomials of x.
def plot_polynomial(y, *args, **kwargs):
    y_values = [y({x: xi}) for xi in x_breaks]
    plt.plot(x_breaks, y_values, *args, **kwargs)
    plt.grid(True)
    plt.xlabel(r'$x$')

# Plot dynamics with zero input.
x_breaks = np.linspace(- xlim, xlim)
plt.figure()
plot_polynomial(f.substitute({u: 0}))
plt.ylabel(r'$f(x, u=0)$')

# Lower bound on the value function (monomial basis)

In [ ]:
def value_function_lower_bound(f, l, X, U, degree):

    # Set up SOS program.
    prog = SosProgram()
    vector_type = type(f.vectors()[0])
    basis = vector_type.construct_basis([x], degree, odd=False)
    J = prog.add_polynomial(basis)[0]

    # Maximize volume beneath the value function.
    Jint = J.definite_integral([x], [- xlim], [xlim])
    prog.add_linear_cost(- Jint.to_scalar())

    # S-procedure for the state limits.
    basis = vector_type.construct_basis([x, u], degree // 2)
    lamx = prog.add_even_sos_polynomial(basis)[0]
    Sprocedure = lamx * X

    # S-procedure for the input limits.
    lamu = prog.add_even_sos_polynomial(basis)[0]
    Sprocedure += lamu * U

    # Bellman inequality.
    Jdot = J.derivative(x) * f
    prog.add_sos_constraint(Jdot + l - Sprocedure)

    # Value function nonpositive in the origin.
    prog.add_linear_constraint(J({x: 0}) <= 0)

    # Solve and retrieve result.
    prog.solve()
    Jlb = prog.substitute_minimizer(J)
    obj = - prog.minimum()

    return Jlb, obj

In [ ]:
# Solve for increasing degree.
degrees = np.arange(1, 11) * 2
Jlb = {d: value_function_lower_bound(f, l, X, U, d) for d in degrees}

In [ ]:
# Plot solution.
plt.figure()
for d in degrees:
    label = f'Deg. {d}, obj. {round(Jlb[d][1], 3)}'
    plot_polynomial(Jlb[d][0], label=label)
    plt.ylabel(r'$J_{\mathrm{lb}}$')
    plt.title('Value-function lower bound (monomial basis)')
    plt.legend()

# Lower bound on the value function (Chebyshev basis)

In [ ]:
# Translate polynomial data in Chebyshev basis.
f_c = f.in_chebyshev_basis()
l_c = l.in_chebyshev_basis()
X_c = X.in_chebyshev_basis()
U_c = U.in_chebyshev_basis()

# Solve for increasing degree.
Jlb_c = {d: value_function_lower_bound(f_c, l_c, X_c, U_c, d) for d in degrees}

In [ ]:
# Plot solution.
plt.figure()
for d in degrees:
    label = f'Deg. {d}, obj. {round(Jlb_c[d][1], 3)}'
    plot_polynomial(Jlb_c[d][0], label=label)
    plt.ylabel(r'$J_{\mathrm{lb}}$')
    plt.title('Value-function lower bound (Chebyshev basis)')
    plt.legend()

# Approximate policy

In [ ]:
# Retrieve approximate policy.
ulb = lambda Jlb: - .5 * R ** (-1) * fu * Jlb.derivative(x)
ulb = {d: ulb(Jlb[d][0]) for d in degrees}

# Plot approximate policy.
plt.figure()
for d in degrees:
    label = f'Deg. {d}'
    plot_polynomial(ulb[d], label=label)
    plt.ylabel(r'$u_{\mathrm{lb}}$')
    plt.title('Approximate policy')
    plt.legend()

# Closed-loop dynamics

In [ ]:
# Derive closed-loop dynamics.
flb = {d: fx + fu * ulb[d] for d in degrees}

# Plot closed-loop dynamics.
plt.figure()
for d in degrees:
    label = f'Deg. {d}'
    plot_polynomial(flb[d], label=label)
    plt.ylabel(r'$f(x, u_{\mathrm{lb}}(x))$')
    plt.title('Closed-loop dynamics')
    plt.legend()

# Bellman inequality

In [ ]:
# Evaluate residuals of Bellman inequality.
llb = lambda ulb: lx + R * ulb ** 2
bellman = lambda Jlb, ulb, flb: llb(ulb) + Jlb.derivative(x) * flb
bellman = {d: bellman(Jlb[d][0], ulb[d], flb[d]) for d in degrees}
llb = {d: llb(ulb[d]) for d in degrees}

# Plot residuals of Bellman inequality.
plt.figure()
for d in degrees:
    label = f'Deg. {d}'
    plot_polynomial(bellman[d], label=label)
    plt.ylabel(r'$l_{\mathrm{lb}}(x) + \dot{J}_{\mathrm{lb}}(x)$')
    plt.title('Residuals of the Bellman inequality')
    plt.yscale('log')
    plt.legend()

# Upper bound on the value function (monomial basis)

In [ ]:
def value_function_upper_bound(degree):

    # Set up SOS program.
    prog = SosProgram()
    vector_type = type(f.vectors()[0])
    basis = vector_type.construct_basis([x], degree, odd=False)
    J = prog.add_polynomial(basis)[0]

    # Minimize volume beneath the value function.
    Jint = J.definite_integral([x], [- xlim], [xlim])
    prog.add_linear_cost(Jint.to_scalar())

    # S-procedure for the state limits.
    basis = vector_type.construct_basis([x, u], degree // 2)
    lamx = prog.add_even_sos_polynomial(basis)[0]
    Sprocedure = lamx * X

    # S-procedure for the input limits.
    lamu = prog.add_even_sos_polynomial(basis)[0]
    Sprocedure += lamu * U

    # Dissipation inequality.
    Jdot = J.derivative(x) * flb[degree]
    prog.add_sos_constraint(- Jdot - llb[degree] - Sprocedure)

    # Value function nonnegative in the origin.
    prog.add_linear_constraint(J({x: 0}) >= 0)

    # Solve and retrieve result.
    prog.solve()
    J_star = prog.substitute_minimizer(J)
    obj = prog.minimum()

    return J_star, obj

In [ ]:
# Solve for increasing degree (only for the stabilized cases).
Jub = {d: value_function_upper_bound(d) for d in degrees[1:]}

In [ ]:
# Plot solution.
plt.figure()
colors = list(mcolors.TABLEAU_COLORS.values())
for i, d in enumerate(degrees[1:]):
    label = f'Deg. {d}'
    plot_polynomial(Jlb[d][0], label=label, c=colors[i+1])
    plot_polynomial(Jub[d][0], c=colors[i+1])
    plt.ylabel(r'$J_{\mathrm{lb}}$, $J_{\mathrm{ub}}$')
    plt.title('Value-function lower and upper bounds')
    plt.legend()